## Aula 12: Funções Customizadas e Otimização com Fusion Kernel

### Parte 1: Cálculo Saxpy

In [15]:
%%writefile saxpy.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <iostream>
#include <chrono>   // Biblioteca para medir o tempo
#include <cstdlib>  // Para a função rand()

// Functor para calcular Saxpy na GPU
struct saxpy
{
    int a;
    saxpy(int a_) : a(a_) {}

    __host__ __device__
    double operator()(const double& x, const double& y) const {
        return a * x + y;
    }
};

// Função para calcular Saxpy na CPU
void saxpy_cpu(const std::vector<double>& x, const std::vector<double>& y, std::vector<double>& z, int a) {
    for (size_t i = 0; i < x.size(); ++i) {
        z[i] = a * x[i] + y[i];
    }
}

int main() {
    const int N = 100000000;  // Tamanho grande para comparação de performance
    int a = 2;

    // Inicializar vetores para a CPU
    std::vector<double> h_x_cpu(N), h_y_cpu(N), h_z_cpu(N);
    for (int i = 0; i < N; ++i) {
        h_x_cpu[i] = rand() % 100;
        h_y_cpu[i] = rand() % 100;
    }

    // Medir tempo para a CPU
    auto start_cpu = std::chrono::high_resolution_clock::now();
    saxpy_cpu(h_x_cpu, h_y_cpu, h_z_cpu, a);
    auto end_cpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duration_cpu = end_cpu - start_cpu;
    std::cout << "Tempo de execução na CPU: " << duration_cpu.count() << " segundos" << std::endl;

    // Inicializar vetores para a GPU
    thrust::host_vector<double> h_x(N), h_y(N), h_z(N);
    for (int i = 0; i < N; ++i) {
        h_x[i] = h_x_cpu[i];
        h_y[i] = h_y_cpu[i];
    }

    // Transferir vetores para a GPU
    thrust::device_vector<double> d_x = h_x;
    thrust::device_vector<double> d_y = h_y;
    thrust::device_vector<double> d_z(N);

    // Medir tempo para a GPU
    auto start_gpu = std::chrono::high_resolution_clock::now();
    thrust::transform(d_x.begin(), d_x.end(), d_y.begin(), d_z.begin(), saxpy(a));
    auto end_gpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duration_gpu = end_gpu - start_gpu;
    std::cout << "Tempo de execução na GPU: " << duration_gpu.count() << " segundos" << std::endl;

    return 0;
}


Overwriting saxpy.cu


In [18]:
!nvcc -arch=sm_75 -std=c++14 saxpy.cu -o saxpy

In [19]:
!./saxpy

Tempo de execução na CPU: 1.05724 segundos
Tempo de execução na GPU: 0.00939285 segundos


### Parte 2: Cálculo da Magnitude de um Vetor

In [23]:
%%writefile magnitude_compare.cu
#include <thrust/device_vector.h>
#include <thrust/transform_reduce.h>
#include <iostream>
#include <vector>
#include <chrono>
#include <cmath>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <cstdlib>  // Para a função rand()

// Functor para elevar ao quadrado
struct square
{
    __host__ __device__
    float operator()(const float& x) const {
        return x * x;
    }
};

// Função para calcular a magnitude na GPU
float magnitude_gpu(thrust::device_vector<float>& v) {
    float sum_of_squares = thrust::transform_reduce(v.begin(), v.end(), square(), 0.0f, thrust::plus<float>());
    return std::sqrt(sum_of_squares);
}

// Função para calcular a magnitude na CPU
float magnitude_cpu(const std::vector<float>& v) {
    float sum_of_squares = 0.0f;
    for (const auto& val : v) {
        sum_of_squares += val * val;
    }
    return std::sqrt(sum_of_squares);
}

int main() {
    const int N = 1000000;  // Tamanho grande para comparação de performance
    std::vector<float> h_v_cpu(N);

    // Inicializar vetor com valores aleatórios
    for (int i = 0; i < N; ++i) {
        h_v_cpu[i] = rand() % 100;
    }

    // Medir tempo para a CPU
    auto start_cpu = std::chrono::high_resolution_clock::now();
    float mag_cpu = magnitude_cpu(h_v_cpu);
    auto end_cpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duration_cpu = end_cpu - start_cpu;
    std::cout << "Tempo de execução na CPU: " << duration_cpu.count() << " segundos" << std::endl;
    std::cout << "Magnitude calculada na CPU: " << mag_cpu << std::endl;

    // Transferir para a GPU
    thrust::host_vector<float> h_v = h_v_cpu;
    thrust::device_vector<float> d_v = h_v;

    // Medir tempo para a GPU
    auto start_gpu = std::chrono::high_resolution_clock::now();
    float mag_gpu = magnitude_gpu(d_v);
    auto end_gpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duration_gpu = end_gpu - start_gpu;
    std::cout << "Tempo de execução na GPU: " << duration_gpu.count() << " segundos" << std::endl;
    std::cout << "Magnitude calculada na GPU: " << mag_gpu << std::endl;

    return 0;
}


Overwriting magnitude_compare.cu


In [26]:
!nvcc -arch=sm_75 -std=c++14 magnitude_compare.cu -o magnitude_compare

In [27]:
!./magnitude_compare

Tempo de execução na CPU: 0.0114911 segundos
Magnitude calculada na CPU: 57344.2
Tempo de execução na GPU: 0.000732848 segundos
Magnitude calculada na GPU: 57297.8


### Parte 3: Fusion Kernel - Cálculo da Variância

In [31]:
%%writefile variance_comparison.cu
#include <thrust/device_vector.h>
#include <thrust/reduce.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <iostream>
#include <chrono>
#include <vector>
#include <cmath>
#include <thrust/host_vector.h>
#include <thrust/transform.h>

// Functor para calcular (x_i - mean)^2 (Fusion Kernel)
struct variance_op
{
    float mean;
    variance_op(float mean_) : mean(mean_) {}

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;
        return diff * diff;
    }
};

// Cálculo de variância com Fusion Kernel
float calculate_variance_gpu(const thrust::device_vector<float>& d_vec, float mean) {
    return thrust::transform_reduce(d_vec.begin(), d_vec.end(), variance_op(mean), 0.0f, thrust::plus<float>()) / d_vec.size();
}

// Cálculo de variância separado (CPU)
float calculate_variance_cpu(const std::vector<float>& vec) {
    float sum = 0.0f;
    for (float v : vec) {
        sum += v;
    }
    float mean = sum / vec.size();

    float variance_sum = 0.0f;
    for (float v : vec) {
        variance_sum += (v - mean) * (v - mean);
    }

    return variance_sum / vec.size();
}

int main() {
    const int N = 1000000;  // Tamanho do vetor

    // Inicializar vetor para CPU e GPU com valores aleatórios
    std::vector<float> vec_cpu(N);
    thrust::host_vector<float> h_vec(N);

    for (int i = 0; i < N; ++i) {
        float val = rand() % 100;
        vec_cpu[i] = val;
        h_vec[i] = val;
    }

    // Transferir os dados para a GPU
    thrust::device_vector<float> d_vec = h_vec;

    // ---------------------- GPU com Fusion Kernel ----------------------
    auto start_gpu = std::chrono::steady_clock::now();

    // Calcular a média na GPU
    float mean_gpu = thrust::reduce(d_vec.begin(), d_vec.end(), 0.0f, thrust::plus<float>()) / N;

    // Calcular a variância na GPU com Fusion Kernel
    float variance_gpu = calculate_variance_gpu(d_vec, mean_gpu);

    auto end_gpu = std::chrono::steady_clock::now();
    std::chrono::duration<double> gpu_time = end_gpu - start_gpu;

    // ---------------------- CPU Cálculo Tradicional ----------------------
    auto start_cpu = std::chrono::steady_clock::now();

    // Calcular a variância na CPU
    float variance_cpu = calculate_variance_cpu(vec_cpu);

    auto end_cpu = std::chrono::steady_clock::now();
    std::chrono::duration<double> cpu_time = end_cpu - start_cpu;

    // Exibir os resultados
    std::cout << "GPU Variância (Fusion Kernel): " << variance_gpu << std::endl;
    std::cout << "GPU Tempo de execução: " << gpu_time.count() << " segundos" << std::endl;

    std::cout << "CPU Variância (Cálculo Tradicional): " << variance_cpu << std::endl;
    std::cout << "CPU Tempo de execução: " << cpu_time.count() << " segundos" << std::endl;

    return 0;
}


Overwriting variance_comparison.cu


In [32]:
!nvcc -arch=sm_75 -std=c++14 variance_comparison.cu -o variance_comparison

In [33]:
!./variance_comparison

GPU Variância (Fusion Kernel): 832.932
GPU Tempo de execução: 0.00101252 segundos
CPU Variância (Cálculo Tradicional): 834.208
CPU Tempo de execução: 0.0230668 segundos


### Análise dos Resultados de Desempenho entre CPU e GPU

Com os resultados obtidos para `n = 1.000.000`, podemos observar uma diferença significativa nos tempos de execução entre as implementações na **CPU** e na **GPU**. Vamos analisar cada parte separadamente e entender as causas das diferenças de tempo, o comportamento esperado para diferentes valores de `n`, e as reflexões sobre os valores calculados.

---

#### **Parte 1: Saxpy**
- **Tempo de execução na CPU**: 1.05724 segundos
- **Tempo de execução na GPU**: 0.00939285 segundos

**Análise:**
A GPU apresentou um tempo de execução muito inferior ao da CPU, cerca de **100 vezes mais rápido**. Isso ocorre porque a GPU foi projetada para processar grandes quantidades de dados de forma paralela. No caso do Saxpy, cada elemento dos vetores pode ser processado independentemente, o que permite que a GPU calcule todos os elementos simultaneamente, aproveitando o paralelismo massivo. Em contrapartida, a CPU, mesmo sendo poderosa, processa os elementos de forma sequencial ou em pequenos lotes (dependendo dos núcleos disponíveis), o que gera um tempo maior para processar um volume de dados tão grande.

**Comportamento com vetores menores:**
Com vetores menores, a diferença entre os tempos de execução na CPU e GPU seria **muito menor**. Isso ocorre porque a GPU exige um **tempo de inicialização** e de **transferência de dados entre a CPU e GPU**, o que se torna uma fração mais significativa do tempo total quando o volume de dados é pequeno. Para pequenos valores de `n`, a CPU pode até ser mais rápida, pois não há necessidade de mover os dados para a GPU, o que elimina a sobrecarga de comunicação.

---

#### **Parte 2: Cálculo da Magnitude**
- **Tempo de execução na CPU**: 0.0114911 segundos
- **Magnitude calculada na CPU**: 57344.2
- **Tempo de execução na GPU**: 0.000732848 segundos
- **Magnitude calculada na GPU**: 57297.8

**Análise:**
Novamente, a GPU apresentou um desempenho muito superior, sendo **mais de 10 vezes mais rápida** que a CPU. A GPU é muito eficiente ao realizar operações paralelas como a elevação ao quadrado de todos os elementos do vetor, enquanto a CPU precisa processar os elementos de forma sequencial.

**Diferença nos valores de magnitude:**
Os valores de magnitude calculados pela CPU e GPU são próximos, mas **não idênticos**. Essa diferença é causada por **pequenas imprecisões numéricas** que ocorrem devido à forma como os números de ponto flutuante são representados e processados em ambas as arquiteturas. A GPU pode utilizar otimizações de precisão, ou processar os valores em uma ordem ligeiramente diferente, o que resulta em essas variações, mas o erro é insignificante para a maioria dos propósitos práticos.

**Comportamento com vetores menores:**
Assim como no caso do Saxpy, para vetores menores, a diferença de tempo seria bem menor. A GPU teria que lidar com a sobrecarga de transferência de dados e inicialização, enquanto a CPU processaria o cálculo de magnitude localmente, sem necessidade de mover os dados entre dispositivos. Para `n` muito pequeno, a CPU pode até ser mais rápida.

---

#### **Parte 3: Cálculo da Variância com Fusion Kernel**
- **Variância calculada na GPU**: 832.932
- **Tempo de execução na GPU**: 0.00101252 segundos
- **Variância calculada na CPU**: 834.208
- **Tempo de execução na CPU**: 0.0230668 segundos

**Análise:**
A diferença de desempenho aqui é **muito grande**, com a GPU sendo **mais de 20 vezes mais rápida**. Isso ocorre porque a técnica de **Fusion Kernel** permite que a GPU combine a transformação e a redução em um único passo, economizando tempo. Na CPU, o cálculo foi feito em duas etapas: primeiro calculando a média, depois somando as diferenças ao quadrado, o que aumenta o tempo total de execução. Na GPU, o **Fusion Kernel** elimina a necessidade de transferir os resultados intermediários entre as operações, resultando em um cálculo extremamente eficiente.

**Diferença nos valores de variância:**
Assim como na magnitude, as pequenas diferenças entre os valores de variância calculados na GPU e na CPU são resultado de **imprecisões numéricas**. A GPU pode processar os valores de ponto flutuante em uma ordem ligeiramente diferente da CPU, o que explica a variação. No entanto, o erro é insignificante para a maioria dos propósitos.

**Comportamento com vetores menores:**
Para vetores menores, o ganho de desempenho da GPU seria menos pronunciado, já que a CPU pode lidar eficientemente com pequenas quantidades de dados sem precisar mover informações para a GPU. Além disso, a inicialização da GPU e a transferência de dados podem representar uma parcela significativa do tempo total de execução para vetores pequenos. Nesse caso, o **Fusion Kernel** ainda seria mais eficiente na GPU em termos de processamento, mas a vantagem em termos de tempo de execução poderia diminuir.

---

### Reflexão sobre os Valores Calculados

- **Comportamento de escalabilidade**: À medida que o tamanho de `n` aumenta, a GPU continua a se destacar em termos de tempo de execução, especialmente em tarefas altamente paralelizáveis, como o cálculo de variância e magnitude. No entanto, para valores pequenos de `n`, a sobrecarga de transferência de dados e inicialização da GPU pode diminuir sua vantagem, ou até torná-la mais lenta que a CPU para esses casos.

### Conclusão

Os resultados confirmam que a GPU é significativamente mais eficiente em termos de tempo para grandes volumes de dados devido à sua capacidade de processamento paralelo massivo. A técnica de **Fusion Kernel** maximiza essa eficiência ao combinar múltiplas operações em uma única etapa. No entanto, para vetores menores, a diferença de desempenho entre CPU e GPU é muito menor, devido à sobrecarga associada ao uso da GPU.

Para `n` grande, a GPU é a melhor opção, enquanto para `n` pequeno, a CPU pode ser mais eficiente devido à menor complexidade de gerenciamento de dados.